# CMR User Configurator

To use this system with its default settings, you will need an account from TACC. If you don't already have a TACC login, please go there now and sign up.

The simplest way to use the User Configurator for the Coastal Model Repository (CMR), is to execute the cells below, one at a time.

In [ ]:
from setvar import *
import os, re, sys

In [ ]:
# Some settings needed by the system
setvar("""
AGAVE_JSON_PARSER=jq
PATH=/usr/local/cli/bin:/usr/bin:/bin:/usr/sbin:/usr/local/sbin:/sbin
""")

## Step 1: Choose your tenant
The tenant is the system that manages your login credentials and Agave data. You
can choose any of the systems below. By default, this notebook uses tacc.prod.

In [ ]:
# In case you don't want to use tacc.prod, here is
# a list of possible Agave tenants.
!tenants-list

### Edit this next line to change the tenant.

In [ ]:
# Set the tenant.
!tenants-init -t tacc.prod

## Step 2: Supply your password

In [ ]:
# Run the cell below to load your AGAVE_PASSWD
# (which will be your TACC password if you use
# the tacc.prod tenant, which is the default).
# By default, this will be the password stored
# in AGAVE_PASSWD.txt. If that file does not exist
# you will be prompted for your password.
#
# If, for some reason, that password is incorrect,
# you can get the system to prompt you again by
# uncommenting the next line and removing the file.
#os.remove("AGAVE_PASSWD.txt")
readpass("AGAVE_PASSWD")

## Step 3: Supply your Username and Email

In [ ]:
# Fill in your Agave username to the right of
# the eqauls sign. (This will be your TACC username
# if you are using the tacc.prod tenant, which is
# the default)
setvar("""
AGAVE_USERNAME=tg457049
EMAIL=sbrandt@cct.lsu.edu
""")

In [ ]:
# This name will be provided for you by the user that runs the
# ResourceConfigurator program.
#setvar("APP_NAME=crcollaboratory-shelob-tg457049-2.0")
setvar("APP_NAME=crcollaboratory-melete05-tg457049-2.0")

## Step 4: Supply the name of the Agave Application you will be using.
You should obtain this from someone who as set up the CMR using the
ResourceConcifguator. By default, jobs will execute on shelob.hpc.lsu.edu
and you will need to contact sbrandt@cct.lsu.edu to gain permission to
run jobs.

## Step 5: Supply your PushBullet Token
Please go to https://www.pushbullet.com/ to obtain your token.

In [ ]:
# Optional:
# The value of your PushBullet token goes here.
# As with the AGAVE_PASSWD entry above, please
# delete it if you wish to use PushBullet to
# receive notifications.
#
#os.remove("PBTOK.txt")
readpass("PBTOK")

# END DATA ENTRY
If you have entered the data above correctly,
you should not need to edit anything below
this line. You will, however, need to execute
the cells below.

In [ ]:
# Some commands to initialize your access to the system
setvar("CLIENT_NAME=user-${AGAVE_USERNAME}")
!clients-delete -u $AGAVE_USERNAME -p $AGAVE_PASSWD $CLIENT_NAME
!clients-create -p $AGAVE_PASSWD -S -N $CLIENT_NAME -u $AGAVE_USERNAME
!auth-tokens-create -u $AGAVE_USERNAME -p $AGAVE_PASSWD

In [ ]:
!apps-list -v ${APP_NAME} > app.txt
import json
with open("app.txt","r") as fd:
    fi = json.load(fd)
    exec_sys=fi["executionSystem"]
    storage_sys=fi["deploymentSystem"]
setvar("""
EXEC_MACHINE={}
STORAGE_MACHINE={}
""".format(exec_sys, storage_sys))

In [ ]:
!systems-list -v ${EXEC_MACHINE} > exec.txt
with open("exec.txt","r") as fd:
    fi = json.load(fd)
    setvar("QUEUE="+fi["queues"][0]["name"])